In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from skimage.io import imread
from skimage.transform import resize
import torch.nn as nn
from torch.optim import Adam
import gc

#constants
grayscale = True
height = 256
width = 256
train1 = {'id':[],'label':[]}
train2 = {'id':[],'label':[]}

#maintain folder variables to switch
ss_images_file = '/content/gdrive/My Drive/Colab Notebooks/ml-for-games-final/Labels/paths_left_right_Lava.csv'
rbr_images_file = '/content/gdrive/My Drive/Colab Notebooks/ml-for-games-mid-term/pressAndpic.txt'
#read input file
with open(ss_images_file,'r') as f:
    ss_images = f.readlines()

with open(rbr_images_file,'r') as f:
    rbr_images = f.readlines()

#split data to feature-label
for i in ss_images:
    a,b = i.split("  ")
    lb = b.replace("\n","")
    train1['id'].append(a)
    train1['label'].append(lb)

for i in rbr_images:
    a,b = i.split("  ")
    lb = b.replace("\n","")
    train2['id'].append(a)
    train2['label'].append(lb)    

#preprocess the image
def preprocess(img):
    img_size = (height, width)
    img = resize(np.array(img), img_size)
    img = np.transpose(img)
    img = img.astype('float32')
    return img
train_img2= []
for img_path in tqdm(train2['id']):
    img = imread(img_path, as_gray=True)
    train_img2.append(preprocess(img))
train_img1 = []
for img_path in tqdm(train1['id']):
    img = imread(img_path, as_gray=True)
    train_img1.append(preprocess(img))



#defining the train data
train_x = np.array(train_img1)
train_x_rbr = np.array(train_img2)
# defining the train label
train_y = np.array(train1['label'])
train_y_rbr = np.array(train2['label'])

#divide into train and validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)
train_x_rbr, val_x_rbr, train_y_rbr, val_y_rbr = train_test_split(train_x_rbr, train_y_rbr, test_size = 0.1)

#set number of epochs
n_epochs = 50   #44 normal and #5 is lab test
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []

# converting the target into torch format
val_x = val_x.reshape(val_x.shape[0], 1,val_x.shape[1],val_x.shape[2])
val_x  = torch.from_numpy(val_x)

val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

x_val, y_val = val_x,val_y
y_val = y_val.type(torch.LongTensor)

val_x_rbr = val_x_rbr.reshape(val_x_rbr.shape[0], 1,val_x_rbr.shape[1],val_x_rbr.shape[2])
val_x_rbr  = torch.from_numpy(val_x_rbr)

val_y_rbr = val_y_rbr.astype(int)
val_y_rbr = torch.from_numpy(val_y_rbr)

x_val_rbr, y_val_rbr = val_x_rbr,val_y_rbr
y_val_rbr = y_val_rbr.type(torch.LongTensor)

#train_y_only_zero = np.array([i for i, v in enumerate(train_y_rbr) if v == 0])
#train_y_only_one = np.array([i for i, v in enumerate(train_y_rbr) if v == 1])

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_layers = nn.Sequential( 
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.cnn_layers2 = nn.Sequential(
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4096,1024),
            nn.Linear(1024,512),
            nn.Linear(512,2),
        )
        self.dropout = nn.Dropout()

    # Defining the forward pass
    def forward(self, x, y = None):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        if not y is None:
            z = self.cnn_layers2(y)
            z = z.view(z.size(0), -1)
            z = z - torch.mean(z)
            x = x + z
        x = self.linear_layers(x)
        return x

model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=3e-4, weight_decay= 1e-5)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

100%|██████████| 9796/9796 [47:38<00:00,  3.43it/s]


In [15]:

def train(epoch, new_x_train,new_x_train_rbr,new_y_train):
    model.train()
    tr_loss = 0
    # converting the data into GPU format
    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(new_x_train_rbr, new_x_train)

    running_corrects_train = 0
    #computing the accuracy
    _, preds_train = torch.max(output_train, 1)
    running_corrects_train += torch.sum(preds_train == new_y_train)

    # computing the training and validation loss
    loss_train = criterion(output_train, new_y_train)
    train_losses.append(loss_train.item())

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    """
    model.eval()
    output_val = model(x_val)
    running_corrects_val = 0
    _, preds_test = torch.max(output_val, 1)
    running_corrects_val += torch.sum(preds_test == y_val)
    loss_val = criterion(output_val, y_val)
    val_losses.append(loss_val.item())

    if epoch % 2 == 0:
        # printing the validation loss
        print('Subway Surfers')
        print('Epoch : ', epoch + 1, '\t', 'loss :', loss_val, 'train_acc :', running_corrects_train.double(), 'val_acc :', running_corrects_val.double())
    """
    model.eval()
    output_val = model(x_val_rbr)
    running_corrects_val = 0
    _, preds_test = torch.max(output_val, 1)
    running_corrects_val += torch.sum(preds_test == y_val_rbr)
    loss_val = criterion(output_val, y_val_rbr)
    val_losses.append(loss_val.item())

    if epoch % 2 == 0:
        # printing the validation loss
        print('Run boy Run')
        print('Epoch : ', epoch + 1, '\t', 'loss :', loss_val, 'train_acc :', running_corrects_train.double(), 'val_acc :', running_corrects_val.double())


x_val = x_val.to('cuda:0')
y_val = y_val.to('cuda:0')
x_val_rbr = x_val_rbr.to('cuda:0')
y_val_rbr = y_val_rbr.to('cuda:0')
min_error = float('+inf')
minima_flag = 0
best_model = Net()
tolerance = 40

batch_size = 64
#set number of epochs
n_epochs = 100   #44 normal and #5 is lab test

# training the model
for epoch in range(n_epochs):
  val_losses = []
  for iteration in range(64):
    indices = np.random.choice(len(train_x),batch_size)
    ipx = torch.from_numpy(train_x[indices].astype(np.float32)).to('cuda:0')
    ipy = torch.from_numpy(train_y[indices].astype(np.long)).to('cuda:0')
    ipx = ipx.view(ipx.shape[0],1,ipx.shape[1],ipx.shape[2])
    indices_rbr = np.random.choice(len(train_x_rbr),batch_size)
    ipx_rbr = torch.from_numpy(train_x_rbr[indices_rbr].astype(np.float32)).to('cuda:0')
    ipy_rbr = torch.from_numpy(train_y_rbr[indices_rbr].astype(np.long)).to('cuda:0')
    ipx_rbr = ipx_rbr.view(ipx_rbr.shape[0],1,ipx_rbr.shape[1],ipx_rbr.shape[2])
    match_indices = {}
    for i, v1 in enumerate(ipy_rbr):
      for j, v2 in enumerate(ipy):
           if v1 == v2:
             if i in match_indices:
               match_indices[i].append(j)
             else:
               match_indices[i] = [j]

    ipx_rbr_select = []
    for k1 in match_indices:
      random_index = np.random.choice(len(match_indices[k1]),1)
      ipx_rbr_select.append(ipx_rbr[random_index[0]])  #Using the random index from 0th arr position
  
    ipx_rbr  = torch.cat(ipx_rbr_select,0)
    ipx_rbr = ipx_rbr.unsqueeze(1)
    train(epoch,ipx,ipx_rbr,ipy)
    del ipx
    del ipy
    del ipx_rbr
    del ipy_rbr
    # gc.collect()
    # torch.cuda.empty_cache()
  mean_iteration_error = np.mean(val_losses)
  if mean_iteration_error < min_error:
    min_error = mean_iteration_error
    best_model = model.cpu().state_dict()
    model.cuda()
    #torch.save(model.cpu().state_dict(), PATH)
    minima_flag = 0
  else:
    minima_flag += 1
    if minima_flag > tolerance:
      break

print(str(min_error))


Streaming output truncated to the last 5000 lines.
Epoch :  1 	 loss : tensor(0.7133, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(63., device='cuda:0', dtype=torch.float64) val_acc : tensor(141., device='cuda:0', dtype=torch.float64)
Run boy Run
Epoch :  1 	 loss : tensor(0.7166, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(64., device='cuda:0', dtype=torch.float64) val_acc : tensor(140., device='cuda:0', dtype=torch.float64)
Run boy Run
Epoch :  1 	 loss : tensor(0.7199, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(64., device='cuda:0', dtype=torch.float64) val_acc : tensor(133., device='cuda:0', dtype=torch.float64)
Run boy Run
Epoch :  1 	 loss : tensor(0.7216, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(63., device='cuda:0', dtype=torch.float64) val_acc : tensor(133., device='cuda:0', dtype=torch.float64)
Run boy Run
Epoch :  3 	 loss : tensor(0.7246, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : te

In [16]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/ml-for-games-final/Zero Shot Learning/model_parallel_0.7062_it64_bs64.pt'
torch.save(best_model, PATH)